In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [2]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

In [3]:
def init_graph_attr(G, AdjFile, df, columns):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    capability_vector = list(zip(*(df[col] for col in columns)))
    node_attri_dict = dict(zip(df["District"], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

In [4]:
G = nx.Graph()
df = pd.read_excel('Agriculture/Agriculture_KAG_2016_17.xlsx')
adjacency_file = 'Karnataka_District_Adjacency_File.xlsx'

# existing_data = pd.DataFrame(df['District'])

existing_data = pd.read_csv('result.csv')

In [5]:
# def addList(l1,l2):
#     for i in range(len(l1)):
#         l1[i] = l1[i] + l2[i]
#     return l1

def addList(l1, l2):
    """
    Element-wise addition of two lists.
    If the lengths are different, append the elements of the shorter list with zeros.
    """
    len_l1, len_l2 = len(l1), len(l2)
    max_len = max(len_l1, len_l2)
    
    # Pad the shorter list with zeros
    l1 = l1 + [0] * (max_len - len_l1)
    l2 = l2 + [0] * (max_len - len_l2)
    
    # Perform element-wise addition
    result = [x + y for x, y in zip(l1, l2)]
    return result

def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1

def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [6]:
def get_node_stress(G,dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [7]:
def get_node_stability(G,dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [8]:
columns = ["Rice_Production_223"]
dim = len(columns)
init_graph_attr(G, "Karnataka_District_Adjacency_File.xlsx", df, columns)

initialstress = get_node_stress(G,dim)
df["Initial Stress"] = df["District"].map(initialstress)

In [9]:
# def calculate_impact_score(df, base_column, capability_vector, intervention):
#     # Perform simple linear regression
#     X = sm.add_constant(df[base_column])
#     y = df[capability_vector]
#     model = sm.OLS(y, X).fit()

#     # Get coefficients and intercept
#     m, c = model.params[base_column], model.params['const']
#     print(c)
#     # Predicted change in capability_vector for base_column + change_percentage% and base_column - change_percentage%
#     base_column_increase = (1 + intervention/100) * df[base_column]
#     base_column_decrease = (1 - intervention/100) * df[base_column]

#     rp_new_plus = m * base_column_increase + c 
#     rp_new_minus = m * base_column_decrease + c 

#     vector_plus = rp_new_plus - m * df[base_column] - c  
#     vector_minus = rp_new_minus - m * df[base_column] - c

#     # Generate normalized values using MinMaxScaler
#     scaler = MinMaxScaler()

#     normalized_vector_plus = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
#     normalized_vector_minus = scaler.fit_transform(vector_minus.values.reshape(-1, 1))

#     # Create new DataFrame with appropriate column names
#     result_df = pd.DataFrame({
#         f'{capability_vector} ({base_column} +{intervention}%)': rp_new_plus,
#         f'{capability_vector} ({base_column} -{intervention}%)': rp_new_minus,
#         f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_vector_plus.flatten(),
#         f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_vector_minus.flatten()
#     })

#     return result_df

In [10]:
# base_Column = "TotalNPK_315"
# CapabilityVector = "Jowar_Yield_278"
# change_percentage = 20

# result_df = calculate_impact_score(df, base_Column, CapabilityVector, change_percentage)

# result_df.head()

In [11]:
def merge_columns_into_dataframe(result_df):
    for column in result_df.columns:
        if column not in existing_data.columns:
            existing_data[column] = result_df[column]
    return existing_data

# merge_columns_into_dataframe(result_df)
# existing_data.head()

In [12]:
# result_df.head()

In [15]:
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import pandas as pd

def calculate_and_map_stability(G, existing_data, adjacency_file, columns_to_pass, intervention, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    taluka_stress_dict = get_node_stability(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability ({intervention}%)"
    stability_column = existing_data["District"].map(taluka_stress_dict)

    return stability_column, stability_column_name
# # Example Usage:
# columns_to_pass_minus20 = ["Normalized Rice_Production_223 (TotalNPK_315 -20%)"]
# columns_to_pass_plus20 = ["Normalized Rice_Production_223 (TotalNPK_315 +20%)"]

# # # Calculate and get stress column for NPK -20%
# # stability_column_minus20, stability_column_name_minus20 = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_minus20, -20, dim)

# # # Calculate and get stress column for NPK +20%
# # stability_column_plus20, stability_column_name_plus20 = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_plus20, 20, dim)

# # # Add stress columns to the result DataFrame
# # existing_data[stability_column_name_plus20] = stability_column_plus20

# # existing_data[stability_column_name_minus20] = stability_column_minus20

# # Display the resulting DataFrame
# existing_data.head()

In [16]:
from bokeh.models import Span, Label

def calculate_and_visualize_impact_stability(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
    # Calculate Impact Score
    X = sm.add_constant(df[base_column])
    y = df[capability_vector]
    model = sm.OLS(y, X).fit()
    
    m, c = model.params[base_column], model.params['const']

    base_column_increase = (1 + intervention/100) * df[base_column]
    base_column_decrease = (1 - intervention/100) * df[base_column]

    change_vector_increase = m * base_column_increase + c
    change_vector_decrease = m * base_column_decrease + c

    scaler_increase = MinMaxScaler()
    normalized_change_increase_vector = scaler_increase.fit_transform(change_vector_increase.values.reshape(-1, 1))

    scaler_decrease = MinMaxScaler()
    normalized_change_decrease_vector = scaler_decrease.fit_transform(change_vector_decrease.values.reshape(-1, 1))


    result_df = pd.DataFrame({
        f'{capability_vector} ({base_column} +{change_percentage}%)': change_vector_increase,
        f'{capability_vector} ({base_column} -{change_percentage}%)': change_vector_decrease,
        f'Normalized {capability_vector} ({base_column} +{change_percentage}%)': normalized_change_increase_vector.flatten(),
        f'Normalized {capability_vector} ({base_column} -{change_percentage}%)': normalized_change_decrease_vector.flatten()
    })

    # Merge Columns
    for column in result_df.columns:
        if column not in existing_data.columns:
            existing_data[column] = result_df[column]

    # Calculate Stability and add to DataFrame
    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if change_percentage >= 0 else '-'}{abs(change_percentage)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, change_percentage, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite change_percentage and add to DataFrame
    opposite_change_percentage = -change_percentage
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_change_percentage >= 0 else '-'}{abs(opposite_change_percentage)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_change_percentage, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    # # Visualize with Bokeh
    # source = ColumnDataSource(existing_data)
    # plot = figure(title=f'Impact vs Stability ({capability_vector} - {base_column} {change_percentage}%)',
    #               y_axis_label=f'Impact ({capability_vector})',
    #               x_axis_label=f'Stability ({base_column} {change_percentage}%)',
    #               tools='pan,box_zoom,wheel_zoom,reset,save')

    # # Plot scatter points
    # plot.scatter(y=f'Normalized {capability_vector} ({base_column} +{change_percentage}%)',
    #             x=stability_column_name,
    #             size=8,
    #             source=source)

    # # Calculate average values from the ColumnDataSource
    # avg_x = source.data[stability_column_name].mean()
    # avg_y = source.data[f'Normalized {capability_vector} ({base_column} +{change_percentage}%)'].mean()

    # # Create Span glyphs for average lines
    # hline = Span(location=avg_y, dimension='width', line_color='red', line_width=2)
    # vline = Span(location=avg_x, dimension='height', line_color='blue', line_width=2)

    # # Add average lines to the plot
    # plot.add_layout(hline)
    # plot.add_layout(vline)

    # # Add labels for average lines
    # plot.add_layout(Label(x=avg_x, y=plot.y_range.end, text=f'Avg {stability_column_name}', text_color='red', text_font_size='10pt', text_align='center'))
    # plot.add_layout(Label(x=plot.x_range.end, y=avg_y, text=f'Avg Normalized {capability_vector}', text_color='blue', text_font_size='10pt', text_align='center'))

    # # Add tooltips
    # hover = HoverTool()
    # hover.tooltips = [("District", "@District")]
    # plot.add_tools(hover)

    # # Display the Bokeh plot in the Jupyter Notebook output cell
    # show(plot, notebook_handle=True)
    return existing_data

base_Column = "TotalNPK_315"
CapabilityVector = "Rice_Production_223"
change_percentage = 20
dim = 1

new_data_1D = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data_1D.head()

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%),New Stability(NPK - 20%),New Stability (20%),New Stability (-20%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.959717,0.959717,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.973330,0.973330,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.853066,0.853066,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.733313,0.733313,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.827968,0.827968,0.827968,0.827968


In [17]:
base_Column = "TotalNPK_315"
CapabilityVector = "Rice_Production_223"
change_percentage = 10
dim = 1

new_data_1D = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
new_data_1D.head()

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%),New Stability(NPK - 20%),Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),New Stability (10%),New Stability (-10%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.959717,0.959717,68479.019211,57507.638667,0.098721,0.098721,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.973330,0.973330,57992.196208,48927.510755,0.075418,0.075418,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.853066,0.853066,27479.191334,23962.324950,0.007616,0.007616,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.733313,0.733313,101029.330849,84139.711825,0.171050,0.171050,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.827968,0.827968,266084.831886,219185.121765,0.537816,0.537816,0.827968,0.827968


In [ ]:
# base_Column = "TotalNPK_315"
# CapabilityVector = "Jowar_Yield_278"
# change_percentage = 20
# dim = 1

# existing_data = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# existing_data.head()

In [ ]:
# Save the resulting DataFrame to a new CSV file if needed
# existing_data.to_csv('result.csv', index=False)

In [ ]:
# base_Column = "TotalNPK_315"
# CapabilityVector = "Rice_Production_223"
# change_percentage = 20
# dim = 1

# existing_data = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# existing_data.head()

In [18]:
import math

def get_node_stability(G, dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0] * dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid, list(G.nodes[nei]["capabilityvector"]))
            except KeyError:
                pass
        
        try:
            # Check if the length of centroid and capabilityvector is the same
            if len(centroid) == dim:
                G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid, len(neighList)), list(G.nodes[n]["capabilityvector"]))
                taluka_stress_dict[G.nodes[n]["name"]] = G.nodes[n]["nodeStress"]
        except KeyError:
            pass

    return taluka_stress_dict

import math

def addList(l1, l2):
    len_l1, len_l2 = len(l1), len(l2)
    max_len = max(len_l1, len_l2)   
    l1 = l1 + [0] * (max_len - len_l1)
    l2 = l2 + [0] * (max_len - len_l2) 
    result = [x + y for x, y in zip(l1, l2)]
    return result

def divList(l, n):
    return [x / n if n != 0 else 0 for x in l]

def l2_normalization(l1, l2):
    len_l1, len_l2 = len(l1), len(l2)
    max_len = max(len_l1, len_l2)
    l1 = l1 + [0] * (max_len - len_l1)
    l2 = l2 + [0] * (max_len - len_l2)
    k = 0
    for i in range(len(l1)):
        k += (l1[i] - l2[i]) ** 2
    return math.sqrt(k)


In [19]:
existing_data.head()

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.827968,0.827968


In [21]:
from bokeh.models import ColumnDataSource, HoverTool, Span, Label
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, save

def calculate_and_visualize_impact_stability(G, existing_data, adjacency_file, base_column, capability_vectors, intervention, dim):
    # print("Columns in existing_data:", existing_data.columns)
    plots = []
    output_file(filename="bokeh_plots.html", title="Static HTML file")

    for capability_vector in capability_vectors:
        # Calculate Impact Score
        X = sm.add_constant(df[base_column])
        y = df[capability_vector]
        model = sm.OLS(y, X).fit()
        m, c = model.params[base_column], model.params['const']
        print(capability_vector)
        print(m)
        print(c)
        base_column_increase = (1 + intervention/100) * df[base_column]
        base_column_decrease = (1 - intervention/100) * df[base_column]

        increased_vector = m * base_column_increase + c
        decreased_vector = m * base_column_decrease + c
        
        change_vector_increase = m * base_column_increase - m * df[base_column]
        change_vector_decrease = m * base_column_decrease - m * df[base_column]

        print(change_vector_increase[0])
        
        scaler_increase = MinMaxScaler()
        normalized_change_increase_vector = scaler_increase.fit_transform(change_vector_increase.values.reshape(-1, 1))
        scaler_decrease = MinMaxScaler()
        normalized_change_decrease_vector = scaler_decrease.fit_transform(change_vector_decrease.values.reshape(-1, 1))

        result_df = pd.DataFrame({
            f'{capability_vector} ({base_column} +{intervention}%)': increased_vector,
            f'{capability_vector} ({base_column} -{intervention}%)': decreased_vector,
            f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_change_increase_vector.flatten(),
            f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_change_decrease_vector.flatten()
        })

        # Merge Columns
        for column in result_df.columns:
            if column not in existing_data.columns:
                existing_data[column] = result_df[column]

        # Calculate Stability and add to DataFrame
        columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
        stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

        existing_data[stability_column_name] = stability_column
        source = ColumnDataSource(existing_data)

        plot_title = f'Impact vs Stability ({capability_vector} - {base_column} {change_percentage}%)'
        y_axis_label = f'Impact ({capability_vector})'
        x_axis_label = f'Stability ({base_column} {change_percentage}%)'

        plot = figure(title=plot_title, y_axis_label=y_axis_label, x_axis_label=x_axis_label,
                      tools='pan,box_zoom,wheel_zoom,reset,save')

        # Plot scatter points
        plot.scatter(y=f'Normalized {capability_vector} ({base_column} +{change_percentage}%)',
                     x=stability_column_name,
                     size=8,
                     source=source,
                     legend_label=f'{capability_vector}')

        # Calculate average values from the ColumnDataSource
        avg_x = source.data[stability_column_name].mean()
        avg_y = source.data[f'Normalized {capability_vector} ({base_column} +{change_percentage}%)'].mean()

        # Create Span glyphs for average lines
        hline = Span(location=avg_y, dimension='width', line_color='red', line_width=2)
        vline = Span(location=avg_x, dimension='height', line_color='blue', line_width=2)

                # Create separate instances of Label for each plot
        hline_label = Label(x=avg_x, y=plot.y_range.end, text=f'Avg {stability_column_name}', text_color='red', text_font_size='10pt', text_align='center')
        vline_label = Label(x=plot.x_range.end, y=avg_y, text=f'Avg Normalized {capability_vector}', text_color='blue', text_font_size='10pt', text_align='center')

        # Add average lines and labels to the plot
        plot.add_layout(hline)
        plot.add_layout(vline)
        plot.add_layout(hline_label)
        plot.add_layout(vline_label)

        # Add tooltips
        hover = HoverTool()
        hover.tooltips = [("District", "@District")]
        plot.add_tools(hover)

        plots.append(plot)

        # Save the individual plot with a unique filename based on the capability vector
        save(plot, filename=f"bokeh_plot_{capability_vector.replace(' ', '_')}.html")

        # Add labels for average lines
        # plot.add_layout(Label(x=avg_x, y=plot.y_range.end, text=f'Avg {stability_column_name}', text_color='red', text_font_size='10pt', text_align='center'))
        # plot.add_layout(Label(x=plot.x_range.end, y=avg_y, text=f'Avg Normalized {capability_vector}', text_color='blue', text_font_size='10pt', text_align='center'))

        # # Add tooltips
        # hover = HoverTool()
        # hover.tooltips = [("District", "@District")]
        # plot.add_tools(hover)

        # plots.append(plot)
        # save(plot)
    layout = gridplot(plots, ncols=2)  # Change ncols based on your preference
    

    # Display the Bokeh layout in the Jupyter Notebook output cell
    show(layout, notebook_handle=True)

    return existing_data

# Example Usage:
base_Column = "TotalNPK_315"
capability_vectors = ["Rice_Production_223", "Maize_Production_226"]
change_percentage = 20
dim = 2

existing_data = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, capability_vectors, change_percentage, dim)

# Display the resulting DataFrame
existing_data.head()


In [22]:
# base_Column = "TotalNPK_315"
# CapabilityVector = ["Rice_Production_223","Maize_Production_226"]
# change_percentage = 20
# dim = 2

# existing_data = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# existing_data.head()